This code has all algorithm which we proposed and we check it using two types of laplacian that is symmetic and unnormalized laplacian. 
And we have set the precision for the edge weigths here so we get desired results for the Petersen graph.
Here we have used edge weigths as (f_i - f_j)^2. 

And we are doing it for just for graphs with edge weigth 1 or 0.

# here i have added the code for random walk laplacian 
# Changed the order in hypothesis as the fielder eigenvector order so that we get kannan algorithm only relevant entries are


In [1]:
import numpy as np
import scipy as sp
from numpy import linalg as LA
import networkx.linalg.algebraicconnectivity as alg
import collections 
import math
import pandas as pd


Results_ncut = pd.DataFrame(index = ['UnL' , 'SymL', 'RwL'])
Results_conductance = pd.DataFrame(index = ['UnL' , 'SymL', 'RwL'])
Graphs = []

In [2]:
# This wil get us the similarity matrix W given the graph and the number of vertices n.
# the vertices are number form 0 to n - 1.
def get_matrix2(graph, n):
    
    w = np.zeros((n, n))
   
    for x in graph.keys() :
        for y in graph[x]:
            w[x][y] = 1
            w[y][x] = 1

    return w
    

In [3]:
def get_unnormalized_laplacian(w):
    
    D = w.sum(axis = 1)
    
    D = np.diag(D)  # The degree matrix D
    
    L = D - w
    
    return L

In [4]:
def get_symmetric_laplacian(w):
    
    D = w.sum(axis = 1)
    D_sqrt = np.sqrt(D)
    
    D_1 = np.reciprocal(D_sqrt)
    D_1 = np.diag(D_1)  
    D = np.diag(D)
    L = np.dot(D_1, np.dot(D - w, D_1))
    
    return L

In [5]:
def get_random_walk_laplacian(w):
    
    D = w.sum(axis = 1)
    
    D_1 = np.reciprocal(D)
    D_1 = np.diag(D_1)  
    D = np.diag(D)
    L = np.dot(D_1, np.dot(D - w, D_1))
    
    return L

In [6]:
# returns the fielder eigen vector given the similartiy matrix W.

def get_fielder(l, n):  #Similairty matix W

    eigvals, eigvecs = sp.linalg.eigh(l)# eigen values and eigen vectors of the unnormalized laplacian D - W 
    #print("Eigenvalues: ",eigvals, "\n")
    eigvecs = np.round( eigvecs, 5)
    eigvals = np.round(eigvals, 4)
    cardinality = 1
    
    fielder_eigval = eigvals[1]
    
    #print(fielder_eigval)
    while(cardinality < n and abs(eigvals[cardinality] - fielder_eigval)<0.000001):
        cardinality = cardinality + 1
        
    fielder = eigvecs.T[1, :] #The 2nd smallest eigen vector of the laplacian L = D - W
    
    return fielder



In [7]:
def ncut(cluster_name, w, n):
    
    cluster_name = np.asarray(cluster_name)
    cluster_name = np.reshape(cluster_name, (1,n))
    
    mull_1 = cluster_name
    mull_2 = np.ones((1,n)) - mull_1
    
    w_1 = np.dot(mull_1, np.dot(w, np.ones((n,1))))
    w_2 = np.dot(mull_2, np.dot(w, np.ones((n,1))))
    
    
    cut = np.dot(mull_1, np.dot(w,mull_2.T))
    #print(cut, w_1,w_2)
    
    ans = cut*(1/w_1 + 1/w_2)
    ans_1 = cut/min(w_1, w_2)
    
    return ans[0][0], ans_1[0][0] 

    

In [8]:
def get_cut(clusters, w, n):
    cut = []
    for x in clusters[0]:
        for y in clusters[1]:
            if(w[x][y] != 0):
                cut.append([x,y])
    return cut

In [9]:
# Here we use our current algorithm to partition the graph
# This returns the clusters, the cluster_name vector and the edeges cut
def get_clusters(f, w, n): # the similarity matrix and the number of vertices
    
    threshold_index = 0
    index = np.argsort(f)
    #print(index)
    
    best_conductance = 2
    cluster_name = [0]*n
    clusters = [[], []]
    
    for i in range(0, n - 1):
        
        cluster_name[index[i]] = 1
        nc, c = ncut(cluster_name, w, n)
        #print(i, nc, c, best_conductance)
        if(c <= best_conductance):
            best_conductance = c
            threshold_index = i
    
    cluster_name = [0]*n
    
    for i in range(0,threshold_index + 1):
        cluster_name[index[i]] = 1
    
    for i in range(0, n):
        clusters[cluster_name[i]].append(i)
        
    # this part changes the cluster_name from a list to numpy array (This step helps to write easy codes)
    cluster_name = np.asarray(cluster_name)
    cluster_name = np.reshape(cluster_name, (1,n))
    
    cp = cluster_name.flatten().tolist()
    
    
    return clusters, cluster_name


In [10]:
def get_results(l, n, w):
    
    
    fielder = get_fielder(l, n)
    add_col_ncut = []
    add_col_con = []
    
    cl, clp = get_clusters(fielder, w, n) 
    nc_1, c_1 = ncut(clp, w, n)
    
    print("Ncut value:", nc_1)
    print("Conductance:", c_1)
    print("Clusters:", cl)
    print("Clusters we get using our current algorithm: ", clp)
    print("The edge-cut of the clusters:", get_cut(cl, w, n))
    
    add_col_ncut.append(nc_1)
    add_col_con.append(c_1)
    
    return add_col_ncut, add_col_con

In [11]:
def get_results_1(l, n, w):
    
    
    fielder = get_fielder(l, n)
    add_col_ncut = []
    add_col_con = []
    
    cl, clp = get_clusters(fielder, w, n) 
    nc_1, c_1 = ncut(clp, w, n)
    
    add_col_ncut.append(nc_1)
    add_col_con.append(c_1)
    
    return add_col_ncut, add_col_con

In [12]:
def unnormalized_results(graph, print_results):

    n = graph[1][0]
    graph = graph[0][0]
    w = get_matrix2(graph, n)

    l = get_unnormalized_laplacian(w)
    
    if(print_results == True):
        print("\nunnormalized results\n")
        return get_results(l, n, w)
    else:
        return get_results_1(l, n, w)

In [13]:
def symmetric_results(graph, print_results):

    n = graph[1][0]
    graph = graph[0][0]
    w = get_matrix2(graph, n)

    l = get_symmetric_laplacian(w)
    
    if(print_results == True):
        print("\nsymmetric results\n")
        return get_results(l, n, w)
    else:
        return get_results_1(l, n, w)

In [14]:
def random_walk_results(graph, print_results):

    n = graph[1][0]
    graph = graph[0][0]
    w = get_matrix2(graph, n)

    l = get_random_walk_laplacian(w)
    
    if(print_results == True):
        print("\nRandomwalk results\n")
        return get_results(l, n, w)
    else:
        return get_results_1(l, n, w)

In [15]:
def show_results(graph, print_results):
    
    add_col_ncut, add_col_con  = [], []
    
    add_col_ncut, add_col_con = symmetric_results(graph[1], print_results)
    un, uc = unnormalized_results(graph[1], print_results)
    rwn, rwc = random_walk_results(graph[1], print_results)
    
    add_col_ncut += un
    add_col_con += uc
    
    add_col_ncut += rwn
    add_col_con += rwc
    
    Results_ncut[graph[0]], Results_conductance[graph[0]] = add_col_ncut, add_col_con
    return 0

In [16]:
Petersen = ['Petersen', [[{0 : [2, 3],
         1 : [4, 3],
         2 : [0, 4],
         3 : [0, 1],
         4 : [1, 2],
         5 : [6, 0],
         6 : [7, 1],
         7 : [8, 2],
         8 : [9, 3],
         9 : [5, 4]
             }],
        [10]]]

Graphs.append(Petersen)
show_results(Petersen, True)


symmetric results

Ncut value: 0.9333333333333333
Conductance: 0.4666666666666667
Clusters: [[0, 4, 5, 8, 9], [1, 2, 3, 6, 7]]
Clusters we get using our current algorithm:  [[0 1 1 1 0 0 1 1 0 0]]
The edge-cut of the clusters: [[0, 2], [0, 3], [4, 1], [4, 2], [5, 6], [8, 3], [8, 7]]

unnormalized results

Ncut value: 0.9333333333333333
Conductance: 0.4666666666666667
Clusters: [[0, 5, 6, 7, 9], [1, 2, 3, 4, 8]]
Clusters we get using our current algorithm:  [[0 1 1 1 1 0 0 0 1 0]]
The edge-cut of the clusters: [[0, 2], [0, 3], [6, 1], [7, 2], [7, 8], [9, 4], [9, 8]]

Randomwalk results

Ncut value: 0.9333333333333333
Conductance: 0.4666666666666667
Clusters: [[2, 5, 7, 8, 9], [0, 1, 3, 4, 6]]
Clusters we get using our current algorithm:  [[1 1 0 1 1 0 1 0 0 0]]
The edge-cut of the clusters: [[2, 0], [2, 4], [5, 0], [5, 6], [7, 6], [8, 3], [9, 4]]


0

In [17]:
Enneahedron = ['Enneahedron', [[{0 : [1, 3, 4],
         1 : [0, 2, 4, 5],
         2 : [1, 3, 6],
         3 : [0, 7, 6, 2],
         4 : [0, 1, 5, 6, 7],
         5 : [1, 4, 6],
         6 : [2, 3, 4, 5, 7],
         7 : [4, 3, 6],
             }],
        [8]]]
Graphs.append(Enneahedron)
show_results(Enneahedron, True)


symmetric results

Ncut value: 0.6666666666666666
Conductance: 0.3333333333333333
Clusters: [[0, 1, 4, 5], [2, 3, 6, 7]]
Clusters we get using our current algorithm:  [[0 0 1 1 0 0 1 1]]
The edge-cut of the clusters: [[0, 3], [1, 2], [4, 6], [4, 7], [5, 6]]

unnormalized results

Ncut value: 0.6666666666666666
Conductance: 0.3333333333333333
Clusters: [[2, 3, 6, 7], [0, 1, 4, 5]]
Clusters we get using our current algorithm:  [[1 1 0 0 1 1 0 0]]
The edge-cut of the clusters: [[2, 1], [3, 0], [6, 4], [6, 5], [7, 4]]

Randomwalk results

Ncut value: 0.6666666666666666
Conductance: 0.3333333333333333
Clusters: [[0, 1, 4, 5], [2, 3, 6, 7]]
Clusters we get using our current algorithm:  [[0 0 1 1 0 0 1 1]]
The edge-cut of the clusters: [[0, 3], [1, 2], [4, 6], [4, 7], [5, 6]]


0

In [18]:
Enneahedron_1 = ['Enneahedron_1',[[{0 : [1, 3, 4],
         1 : [0, 2, 4, 6, 5],
         2 : [1, 3, 5],
         3 : [0, 4, 7, 5, 2],
         4 : [0, 1, 3, 6],
         5 : [1, 7, 3, 2],
         6 : [1, 3, 4, 7],
         7 : [6, 5],
             }],
        [8]]]
Graphs.append(Enneahedron_1)
show_results(Enneahedron_1, True)


symmetric results

Ncut value: 0.75
Conductance: 0.375
Clusters: [[0, 1, 4, 6], [2, 3, 5, 7]]
Clusters we get using our current algorithm:  [[0 0 1 1 0 1 0 1]]
The edge-cut of the clusters: [[0, 3], [1, 2], [1, 5], [4, 3], [6, 3], [6, 7]]

unnormalized results

Ncut value: 0.75
Conductance: 0.375
Clusters: [[0, 1, 4, 6], [2, 3, 5, 7]]
Clusters we get using our current algorithm:  [[0 0 1 1 0 1 0 1]]
The edge-cut of the clusters: [[0, 3], [1, 2], [1, 5], [4, 3], [6, 3], [6, 7]]

Randomwalk results

Ncut value: 0.75
Conductance: 0.375
Clusters: [[0, 1, 4, 6], [2, 3, 5, 7]]
Clusters we get using our current algorithm:  [[0 0 1 1 0 1 0 1]]
The edge-cut of the clusters: [[0, 3], [1, 2], [1, 5], [4, 3], [6, 3], [6, 7]]


0

In [19]:
Cockroach = ['Cockroach',[[{0 : [1],
         1 : [0, 2],
         2 : [1, 3],
         3 : [2, 4, 8],
         4 : [3, 5, 7],
         5 : [4, 6],
         6 : [5, 7],
         7 : [6, 8, 4],
         8 : [7, 9, 3],
         9 : [8, 10],
         10 : [11]
             }],
        [12]]]
Graphs.append(Cockroach)
show_results(Cockroach, True)


symmetric results

Ncut value: 0.24761904761904763
Conductance: 0.2
Clusters: [[0, 1, 2], [3, 4, 5, 6, 7, 8, 9, 10, 11]]
Clusters we get using our current algorithm:  [[0 0 0 1 1 1 1 1 1 1 1 1]]
The edge-cut of the clusters: [[2, 3]]

unnormalized results

Ncut value: 0.24761904761904763
Conductance: 0.2
Clusters: [[0, 1, 2], [3, 4, 5, 6, 7, 8, 9, 10, 11]]
Clusters we get using our current algorithm:  [[0 0 0 1 1 1 1 1 1 1 1 1]]
The edge-cut of the clusters: [[2, 3]]

Randomwalk results

Ncut value: 0.24761904761904763
Conductance: 0.2
Clusters: [[0, 1, 2], [3, 4, 5, 6, 7, 8, 9, 10, 11]]
Clusters we get using our current algorithm:  [[0 0 0 1 1 1 1 1 1 1 1 1]]
The edge-cut of the clusters: [[2, 3]]


0

In [20]:
Cockroach_1 = ['Cockroach_1', [[{0 : [1, 11],
         1 : [0, 2],
         2 : [1, 3],
         3 : [2, 4, 8],
         4 : [3, 5, 7],
         5 : [4, 6],
         6 : [5, 7],
         7 : [6, 8, 4],
         8 : [7, 9, 3],
         9 : [8, 10],
         10 : [11],
         11 : [0]    }],
        [12]]]

Graphs.append(Cockroach_1)
show_results(Cockroach_1, True)


symmetric results

Ncut value: 0.29166666666666663
Conductance: 0.16666666666666666
Clusters: [[3, 4, 5, 6, 7, 8], [0, 1, 2, 9, 10, 11]]
Clusters we get using our current algorithm:  [[1 1 1 0 0 0 0 0 0 1 1 1]]
The edge-cut of the clusters: [[3, 2], [8, 9]]

unnormalized results

Ncut value: 0.29166666666666663
Conductance: 0.16666666666666666
Clusters: [[3, 4, 5, 6, 7, 8], [0, 1, 2, 9, 10, 11]]
Clusters we get using our current algorithm:  [[1 1 1 0 0 0 0 0 0 1 1 1]]
The edge-cut of the clusters: [[3, 2], [8, 9]]

Randomwalk results

Ncut value: 0.29166666666666663
Conductance: 0.16666666666666666
Clusters: [[0, 1, 2, 9, 10, 11], [3, 4, 5, 6, 7, 8]]
Clusters we get using our current algorithm:  [[0 0 0 1 1 1 1 1 1 0 0 0]]
The edge-cut of the clusters: [[2, 3], [9, 8]]


0

In [21]:
Path_point  = ['Path_point',[[{0 : [1],
         1 : [0, 2],
         2 : [1, 3],
         3 : [2, 4],
         4 : [3, 5],
         5 : [4, 6],
         6 : [5, 7],
         7 : [6, 8],
         8: [7, 9],
         9: [8, 10],
         10:[9, 11],
         11 :[0,1,2,3,4,5,6,7,8,9,10]
                }],
        [12]]]

Graphs.append(Path_point)
show_results(Path_point, True)


symmetric results

Ncut value: 0.6917647058823528
Conductance: 0.4117647058823529
Clusters: [[0, 1, 2, 3, 4, 11], [5, 6, 7, 8, 9, 10]]
Clusters we get using our current algorithm:  [[0 0 0 0 0 1 1 1 1 1 1 0]]
The edge-cut of the clusters: [[4, 5], [11, 5], [11, 6], [11, 7], [11, 8], [11, 9], [11, 10]]

unnormalized results

Ncut value: 0.6917647058823528
Conductance: 0.4117647058823529
Clusters: [[0, 1, 2, 3, 4, 11], [5, 6, 7, 8, 9, 10]]
Clusters we get using our current algorithm:  [[0 0 0 0 0 1 1 1 1 1 1 0]]
The edge-cut of the clusters: [[4, 5], [11, 5], [11, 6], [11, 7], [11, 8], [11, 9], [11, 10]]

Randomwalk results

Ncut value: 0.6917647058823528
Conductance: 0.4117647058823529
Clusters: [[0, 1, 2, 3, 4, 11], [5, 6, 7, 8, 9, 10]]
Clusters we get using our current algorithm:  [[0 0 0 0 0 1 1 1 1 1 1 0]]
The edge-cut of the clusters: [[4, 5], [11, 5], [11, 6], [11, 7], [11, 8], [11, 9], [11, 10]]


0

In [22]:
Two_clusters = ['Two_clusters', [[{0 : [1, 2, 3],
         1 : [0, 2, 3, 4],
         2 : [0, 1, 3],
         3 : [0, 1, 2],
         4 : [5, 6, 7],
         5 : [4, 6, 7],
         6 : [4, 5, 7],
         7 : [4, 5, 6]
        }],
        [8]]]

Graphs.append(Two_clusters)
show_results(Two_clusters, True)


symmetric results

Ncut value: 0.15384615384615385
Conductance: 0.07692307692307693
Clusters: [[4, 5, 6, 7], [0, 1, 2, 3]]
Clusters we get using our current algorithm:  [[1 1 1 1 0 0 0 0]]
The edge-cut of the clusters: [[4, 1]]

unnormalized results

Ncut value: 0.15384615384615385
Conductance: 0.07692307692307693
Clusters: [[4, 5, 6, 7], [0, 1, 2, 3]]
Clusters we get using our current algorithm:  [[1 1 1 1 0 0 0 0]]
The edge-cut of the clusters: [[4, 1]]

Randomwalk results

Ncut value: 0.15384615384615385
Conductance: 0.07692307692307693
Clusters: [[0, 1, 2, 3], [4, 5, 6, 7]]
Clusters we get using our current algorithm:  [[0 0 0 0 1 1 1 1]]
The edge-cut of the clusters: [[1, 4]]


0

In [23]:
Discrete_moons = {}

for i in range(0, 19):
    Discrete_moons[i] = [i + 1]
    Discrete_moons[i + 20] = [i + 21]

print(Discrete_moons)

for i in range(0, 10):
    Discrete_moons[i].append(19 - i)
    Discrete_moons[i + 20].append(39 - i)

for i in range(0, 5):
    Discrete_moons[i + 10].append(24 - i)
#print(Discrete_moons)

D_M = ['Discrete_moons',[[Discrete_moons], [40]]]

Graphs.append(D_M)
show_results(D_M, True)

{0: [1], 20: [21], 1: [2], 21: [22], 2: [3], 22: [23], 3: [4], 23: [24], 4: [5], 24: [25], 5: [6], 25: [26], 6: [7], 26: [27], 7: [8], 27: [28], 8: [9], 28: [29], 9: [10], 29: [30], 10: [11], 30: [31], 11: [12], 31: [32], 12: [13], 32: [33], 13: [14], 33: [34], 14: [15], 34: [35], 15: [16], 35: [36], 16: [17], 36: [37], 17: [18], 37: [38], 18: [19], 38: [39]}

symmetric results

Ncut value: 0.09270516717325228
Conductance: 0.07142857142857142
Clusters: [[25, 26, 27, 28, 29, 30, 31, 32, 33, 34], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 35, 36, 37, 38, 39]]
Clusters we get using our current algorithm:  [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1
  1 1 1 1]]
The edge-cut of the clusters: [[25, 24], [34, 35]]

unnormalized results

Ncut value: 0.09270516717325228
Conductance: 0.07142857142857142
Clusters: [[25, 26, 27, 28, 29, 30, 31, 32, 33, 34], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1

0

In [24]:
Double_Tree = {}

for i in range(1,10):
    Double_Tree[i] = [math.floor((i - 1)/2)]
    Double_Tree[i + 10] = [10 + math.floor((i - 1)/2)]

Double_Tree[0] = [10]

D_T = ['Double_Tree',[[Double_Tree], [20]]]

Graphs.append(D_T)
show_results(D_T, True)


symmetric results

Ncut value: 0.10526315789473684
Conductance: 0.05263157894736842
Clusters: [[10, 11, 12, 13, 14, 15, 16, 17, 18, 19], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]
Clusters we get using our current algorithm:  [[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]]
The edge-cut of the clusters: [[10, 0]]

unnormalized results

Ncut value: 0.10526315789473684
Conductance: 0.05263157894736842
Clusters: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]
Clusters we get using our current algorithm:  [[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]]
The edge-cut of the clusters: [[0, 10]]

Randomwalk results

Ncut value: 0.10526315789473684
Conductance: 0.05263157894736842
Clusters: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]
Clusters we get using our current algorithm:  [[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]]
The edge-cut of the clusters: [[0, 10]]


0

In [25]:
print(D_M)

['Discrete_moons', [[{0: [1, 19], 20: [21, 39], 1: [2, 18], 21: [22, 38], 2: [3, 17], 22: [23, 37], 3: [4, 16], 23: [24, 36], 4: [5, 15], 24: [25, 35], 5: [6, 14], 25: [26, 34], 6: [7, 13], 26: [27, 33], 7: [8, 12], 27: [28, 32], 8: [9, 11], 28: [29, 31], 9: [10, 10], 29: [30, 30], 10: [11, 24], 30: [31], 11: [12, 23], 31: [32], 12: [13, 22], 32: [33], 13: [14, 21], 33: [34], 14: [15, 20], 34: [35], 15: [16], 35: [36], 16: [17], 36: [37], 17: [18], 37: [38], 18: [19], 38: [39]}], [40]]]


In [26]:
Double_Tree1 = {}

for i in range(1,10):
    Double_Tree1[i] = [math.floor((i - 1)/2)]
    Double_Tree1[i + 10] = [10 + math.floor((i - 1)/2)]

Double_Tree1[0] = [10]

for i in range(5,9):
     Double_Tree1[i].append(10+i)
for i in range(0,9):
    Double_Tree1[i].append((i+1)%10)
D_T1 = ['Double_Tree1',[[Double_Tree1], [20]]]

Graphs.append(D_T1)
print(D_T1)
show_results(D_T1, True)



['Double_Tree1', [[{1: [0, 2], 11: [10], 2: [0, 3], 12: [10], 3: [1, 4], 13: [11], 4: [1, 5], 14: [11], 5: [2, 15, 6], 15: [12], 6: [2, 16, 7], 16: [12], 7: [3, 17, 8], 17: [13], 8: [3, 18, 9], 18: [13], 9: [4], 19: [14], 0: [10, 1]}], [20]]]

symmetric results

Ncut value: 0.33695652173913043
Conductance: 0.25
Clusters: [[10, 11, 13, 14, 17, 18, 19], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 15, 16]]
Clusters we get using our current algorithm:  [[1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 0 0]]
The edge-cut of the clusters: [[10, 0], [10, 12], [17, 7], [18, 8]]

unnormalized results

Ncut value: 0.33695652173913043
Conductance: 0.25
Clusters: [[10, 11, 13, 14, 17, 18, 19], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 15, 16]]
Clusters we get using our current algorithm:  [[1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 0 0]]
The edge-cut of the clusters: [[10, 0], [10, 12], [17, 7], [18, 8]]

Randomwalk results

Ncut value: 0.4535005224660397
Conductance: 0.2413793103448276
Clusters: [[7, 8, 10, 11, 12, 13, 14, 16, 17

0

In [27]:
for graph in Graphs:
    show_results(graph, False)
    
    

In [28]:
Results_conductance.T


,UnL,SymL,RwL
Petersen,0.466667,0.466667,0.466667
Enneahedron,0.333333,0.333333,0.333333
Enneahedron_1,0.375000,0.375000,0.375000
Cockroach,0.200000,0.200000,0.200000
Cockroach_1,0.166667,0.166667,0.166667
Path_point,0.411765,0.411765,0.411765
Two_clusters,0.076923,0.076923,0.076923
Discrete_moons,0.071429,0.071429,0.071429
Double_Tree,0.052632,0.052632,0.052632
Double_Tree1,0.250000,0.250000,0.241379


In [29]:
Results_ncut.T

,UnL,SymL,RwL
Petersen,0.933333,0.933333,0.933333
Enneahedron,0.666667,0.666667,0.666667
Enneahedron_1,0.750000,0.750000,0.750000
Cockroach,0.247619,0.247619,0.247619
Cockroach_1,0.291667,0.291667,0.291667
Path_point,0.691765,0.691765,0.691765
Two_clusters,0.153846,0.153846,0.153846
Discrete_moons,0.092705,0.092705,0.092705
Double_Tree,0.105263,0.105263,0.105263
Double_Tree1,0.336957,0.336957,0.453501
